# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. 
<br>Входные данные от банка — статистика о платёжеспособности клиентов.
<br>Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<b>Цель исследования:<b>

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
    
<b>Ход исследования:<b>
    <br>- Предобработка данных
    <br>- Декомпозиция исходного датафрейма
    <br>- Категоризация данных
    <br>- Формирование заключений

<b>Описание данных:<b>
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

Получение общей информации о данных в датасете.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     19351 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      19351 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


В данных обнаружены пропуски в столбцах 'days_employed' и 'total_income'.
Изучим первые 10 строк датасета.

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце 'days_employed' помимо пропусков присутствуют аномальные данные: отрицательные и абсурдно высокие значения.

### Шаг 2.1 Заполнение пропусков

Предобработку данных начнём с заполнения пропущенных значений в столбце с ежемесячным доходом 'total_income'.

Ежемесячный доход пропущен в 2174 случаях - это 10% всех значений датафрейма. 
Соответственно, просто "пожертвовать" данными из строк, в которых нет значений, мы не можем.
Для начала отфильтруем строки с пропущенными значениями чтобы попробовать выявить закономерности.

In [3]:
data[data['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Закономерности в пропусках не обнаружены. Заполним их медианным значением, чтобы результат обработки не исказил данные. И выведем обновлённую общую информацию о датасете.

In [4]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     19351 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


Причин пропущенных значений в столбце с ежемесячным доходом может быть несколько:
- Сотруднику банка было достаточно предоставленных данных для принятия решения;
- Заёмщик решил скрыть данную информацию;
- Сотрудник банка при занесении данных пропустил этот параметр;
- Произошёл сбой системы и данные за определённый период по двум столбцам таблицы "слетели".

Пропущенные значения в столбце 'total_income' обработаны. В столбце 'days_employed' нужно сначала разобраться с аномалиями.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Отрицательные значения в столбце 'days_employed' с общим трудовым стажем могли возникнуть в следствии того, что были перепутаны даты начала и конца работы. Завышенные значения могли появиться, если при внесении данных перепутать единицы измерения (вместо дней внести часы).
Объединим данные в категории:
- Клиенты до 18 лет включительно попадают в категорию «дети»;
- Клиенты от 19 до 64 лет — категория «взрослые»;
- Клиенты 65 лет и старше принадлежат к категории «пенсионеры».

In [5]:
def age_group(age):
    """
    Возвращает возрастную группу по значению возраста age, используя правила:
    - 'дети', если age <= 18 лет;
    - 'взрослые', если age от 19 до 64;
    - 'пенсионеры' — от 65 и старше.
    """
    try:
        if age <= 18:
            return 'дети'
        if age <= 64:
            return 'взрослые'
        return 'пенсионеры'
    except:
        display('Некорректное указание возраста')

#Проверка работоспособности функции

display(age_group(14))
display(age_group('cat'))
display(age_group(99))


'дети'

'Некорректное указание возраста'

None

'пенсионеры'

In [6]:
data['age_group'] = data['dob_years'].apply(age_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослые
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослые
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,взрослые
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослые
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослые


Выведем на экран количество клиентов каждой возрастной категории.

In [7]:
data.groupby('age_group').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
age_group,,,,,,,,,,,,
взрослые,20525,18442,20525,20525,20525,20525,20525,20525,20525,20525,20525,20525
дети,101,91,101,101,101,101,101,101,101,101,101,101
пенсионеры,899,818,899,899,899,899,899,899,899,899,899,899


Посмотрим медианные значения в столбце days_employed по каждой возрастной группе.

In [8]:
display('Медианное значение в возрастной группе "дети":', data[data['age_group'] == 'дети']['days_employed'].median())
display('Медианное значение в возрастной группе "взрослые":', data[data['age_group'] == 'взрослые']['days_employed'].median())
display('Медианное значение в возрастной группе "пенсионеры":', data[data['age_group'] == 'пенсионеры']['days_employed'].median())

'Медианное значение в возрастной группе "дети":'

-1146.6895862897618

'Медианное значение в возрастной группе "взрослые":'

-1277.251538081112

'Медианное значение в возрастной группе "пенсионеры":'

360304.23230829864

Заполнение данными значениями не очень корректное действие. Но по заданию нет использования данных из столбца с общим стажем в дальнейшем. Если бы такое потребовалось, то рекомендовано обратиться к поставщику данных с уточнениями - для корректного избавления от пропусков и аномалий. Также можно удалить вновь созданный столбец 'age_group', так как он для нас в этот раз не актуален.

In [9]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data = data.drop(['age_group'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     21525 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


Пропущенных значений в таблице нет.

Проверка аномальных значений в столбце с количеством детей.

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Отрицательным количество быть не может априори. Необходимо заменить данные медианным значением.

In [11]:
data['children'] = data['children'].replace(-1, data['children'].median()).astype(int)
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Проверим возраст людей, в семьях которых количество детей равно двадцати.

In [12]:
data[data['children'] == 20]['dob_years']

606      21
720      44
1074     56
2510     59
2941      0
         ..
21008    40
21325    37
21390    53
21404    52
21491    27
Name: dob_years, Length: 76, dtype: int64

В датасете таких семей всего 76, что составляет 0,4% от общего количества. Их можно проигнорировать.
    
В ходе этой проверки замечены заёмщики с нулевым возрастом.
Проверим уникальные значения в столбце с возрастом.

In [13]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Подсчёт количества заёмщицов, у которых указан нулевой возраст.

In [14]:
data[data['dob_years'] == 0]['children'].count()

101

По поводу количества детей, равного 20, можно только предполагать: либо это действительно очень большие семьи (в таком случае это будет исключением из правила), либо в данных опечатка (и на самом деле, там должно быть 2 ребёнка, например).
<br>Заёмщики с нулевым возрастом, возможно, не пожелали указывать свой настоящий возраст, либо эта информация не требовалась при заполнении данных для выдачи кредита. В нашем исследовании информация о возрасте в дальнейшем не потребуется. Можно с этими данными ничего не делать.

### Шаг 2.3. Изменение типов данных.

Замена вещественного типа значений в столбце 'total_income' на целочисленный.

In [15]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     21525 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  int64  

 11  purpose           21525 non-null  object 

dtypes: float64(1), int64(6), object(5)

memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Поиск неявных дубликатов в столбцах.

In [16]:
display(data['education'].unique())
display(data['family_status'].unique())
display(data['gender'].unique())
display(data['income_type'].unique())
display(data['purpose'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['F', 'M', 'XNA'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Приведение данных в столбце 'education' к единообразному виду.

In [17]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим датасет на наличие явных дубликатов.

In [18]:
data.duplicated().sum()

71

Удалим явные дубликаты из таблицы.

In [19]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

От неявных дубликатов в столбце 'education' избавились методом .str.lower(), так как данные различались только регистром. Такие дубликаты могли явиться следствием человеческого фактора: зажатый 'Caps Lock', привычка начинать каждую новую строку с заглавной буквы.

Явные дубликаты могли появиться в следствии человеческого фактора (случайное копирование строк, например).

Теперь и от них датафрейм чист.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создание таблицы 'education_log', в которой каждому уникальному значению из 'education' соответствует уникальное значение 'education_id'.

In [20]:
education_log = data[['education', 'education_id']]
education_log = education_log.drop_duplicates().reset_index(drop=True)
education_log

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создание таблицы 'family_log', в которой каждому уникальному значению из 'family_status' соответствует уникальное значение 'family_status_id'.

In [21]:
family_log = data[['family_status', 'family_status_id']]
family_log = family_log.drop_duplicates().reset_index(drop=True)
family_log

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удаление из исходного датафрейма столбцов 'education' и 'family_status', оставим только их идентификаторы: education_id и family_status_id. В качестве "словарей", к которым можно обращаться по идентификатору будут использованы 'education_log' и 'family_log'.

In [22]:
data = data.drop(['education', 'family_status'], axis=1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Создание столбца 'total_income_category' с категориями ежемесячного дохода:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'

Первый этап: создание функции и проверка её работоспособности.

In [23]:
def total_income_category(total_income):
    try:
        if total_income <= 30000:
            return 'E'
        if total_income <= 50000:
            return 'D'
        if total_income <= 200000:
            return 'C'
        if total_income <= 1000000:
            return 'B'
        return 'A'
    except:
        display('Некорректное значение дохода')
        
display(total_income_category(15000))
display(total_income_category('4o5000'))
display(total_income_category(100000))
display(total_income_category(500000))
display(total_income_category(2000000))

'E'

'Некорректное значение дохода'

None

'C'

'B'

'A'

Второй этап: добавление столбца к датафрейму. И вывод его первых строк.

In [24]:
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Создание столбца 'purpose_category', в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Первый этап: написание функции и проверка её работоспособности.

In [25]:
def purpose_category(purpose):
    try:
        if 'авто' in purpose:
            return 'операции с автомобилем'
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'образ' in purpose:
            return 'получение образования'
        return 'операции с недвижимостью'
    except:
        display('Некорректное значение')
    
display(purpose_category('высшее образование'))
display(purpose_category('жилье'))
display(purpose_category('на проведение свадьбы'))
display(purpose_category('приобретение автомобиля'))

'получение образования'

'операции с недвижимостью'

'проведение свадьбы'

'операции с автомобилем'

Второй этап: добавление столбца к датафрейму и вывод его первых строк.

In [26]:
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [27]:
#Количество задержек по возврату кредитов, сгруппированное по количеству детей.
debt_by_children_sum = data.groupby('children')['debt'].sum()

#Общее количество выданных кредитов по категориям.
debt_children_count = data.groupby('children')['debt'].count()

#Доля просроченных кредитов в выданных по категориям.
debt_by_children_sum / debt_children_count *100

children
0      7.525817
1      9.234609
2      9.454191
3      8.181818
4      9.756098
5      0.000000
20    10.526316
Name: debt, dtype: float64

In [28]:
#То же самое с использованием сводной таблицы:

data.pivot_table(index=['children'], values=['debt'], aggfunc='mean')

,debt
children,
0,0.075258
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


Проверим количество невозвратов кредитов по отношению к общему количеству займов, отфильтрованное по количеству детей.

In [29]:
debt_by_children_sum / 21454 * 100

children
0     4.959448
1     2.069544
2     0.904260
3     0.125851
4     0.018645
5     0.000000
20    0.037289
Name: debt, dtype: float64

##### Вывод 1:
Заёмщики без детей имеют тенденцию к просрочке возращения кредита чаще, чем заёмщики с детьми. При наличии в семье от двух детей и более, заёмщики практически всегда выплачивают кредиты в срок.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
#Количество задержек по возврату кредитов, сгруппированно по семейному положению.
debt_by_family_status_sum = data.merge(family_log, on='family_status_id', how='left').groupby('family_status')['debt'].sum()

#Общее количество выданных кредитов по категориям.
debt_by_family_status_count = data.merge(family_log, on='family_status_id', how='left').groupby('family_status')['debt'].count()

#Доля просроченных кредитов в выданных по категориям.
debt_by_family_status_sum / debt_by_family_status_count * 100

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64

Проверим количество невозвратов кредитов по отношению к общему количеству займов, отфильтрованное по семейному положению.

In [31]:
debt_by_family_status_sum / 21454 *100

family_status
Не женат / не замужем    1.277151
в разводе                0.396197
вдовец / вдова           0.293652
гражданский брак         1.808521
женат / замужем          4.339517
Name: debt, dtype: float64

##### Вывод 2: 
Из общего числа заёмщиков люди, состоящие в браке, возвращают кредиты хуже остальных категорий. Одинокие заёмщики, ранее состоящие в браке, возвращают вовремя заёмные средства практически в 100% случаев.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
#Количество задержек по возврату кредитов, сгруппированно по семейному положению.
debt_by_total_income_category_sum = data.groupby('total_income_category')['debt'].sum()

#Общее количество выданных кредитов по категориям.
debt_by_total_income_category_count = data.groupby('total_income_category')['debt'].count()

#Доля просроченных кредитов в выданных по категориям.
debt_by_total_income_category_sum / debt_by_total_income_category_count * 100

total_income_category
A    8.000000
B    7.062091
C    8.491508
D    6.000000
E    9.090909
Name: debt, dtype: float64

Проверим количество невозвратов кредитов по отношению к общему количеству займов, отфильтрованное по категориям ежемесячного дохода.

In [33]:
debt_by_total_income_category_sum / 21454 *100

total_income_category
A    0.009322
B    1.659364
C    6.339144
D    0.097884
E    0.009322
Name: debt, dtype: float64

##### Вывод 3:
Заёмщики с доходом от 50000 до 200000 позволяют себе вернуть кредит с просрочкой в 6% случаев от общего количества выданных кредитов.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [34]:
#Количество задержек по возврату кредитов, сгруппированно по цели кредита.
debt_by_purpose_category_sum = data.groupby('purpose_category')['debt'].sum()

#Общее количество выданных кредитов по категориям.
debt_by_purpose_category_count = data.groupby('purpose_category')['debt'].count()

#Доля просроченных кредитов в выданных по категориям.
debt_by_purpose_category_sum / debt_by_purpose_category_count * 100

purpose_category
операции с автомобилем      9.359034
операции с недвижимостью    7.233373
получение образования       9.220035
проведение свадьбы          8.003442
Name: debt, dtype: float64

Проверим количество невозвратов кредитов по отношению к общему количеству займов, по цели оформления кредита.

In [35]:
debt_by_purpose_category_sum / 21454 * 100

purpose_category
операции с автомобилем      1.878438
операции с недвижимостью    3.645008
получение образования       1.724620
проведение свадьбы          0.866971
Name: debt, dtype: float64

##### Вывод 4:
Заёмщики, оформившие кредит на операции с недвижимостью в 3% случаев получения кредитов возвращают их несвоевременно. Лучше всех возвращаю вредиты, люди, взявшие их на проведение свадьбы.

## Общий вывод:

Проведя исследование по влиянию разных факторов на возврат кредита в срок, подвожу итоги:
- Заёмщики без детей имеют тенденцию к просрочке возращения кредита чаще, чем заёмщики с детьми. При наличии в семье от двух детей и более, заёмщики практически всегда выплачивают кредиты в срок.
- Из общего числа заёмщиков люди, состоящие в браке, возвращают кредиты хуже остальных категорий. Одинокие заёмщики, ранее состоящие в браке, возвращают вовремя заёмные средства практически в 100% случаев.
- Заёмщики с доходом от 50000 до 200000 позволяют себе вернуть кредит с просрочкой в 6% случаев от общего количества выданных кредитов. Остальные категории заёмщиков по уровню дохода возвращают займы практически одинаково хорошо.
- Заёмщики, оформившие кредит на операции с недвижимостью в 3% случаев получения кредитов возвращают их несвоевременно. Лучше всех возвращаю вредиты, люди, взявшие их на проведение свадьбы.